In [24]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-06-29 11:56:27--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8002::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.14’

input.txt.14        100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2023-06-29 11:56:28 (10.7 MB/s) - ‘input.txt.14’ saved [1115394/1115394]



In [25]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [26]:
len(text)

1115394

In [27]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [28]:
vocab = sorted(list(set(text)))
vocab_size = len(vocab)

In [29]:
print("".join(vocab), vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz 65


In [30]:
# Create a Mapping -- simple char to int mapping
# Map str to int -- stoi
stoi, itos = {}, {}
for i, ch in enumerate(vocab):
    stoi[ch] = i
for i, ch in enumerate(vocab):
    itos[i] = ch

In [31]:
print(stoi, itos)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64} {0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i',

In [32]:
# Encoding and Decoding
# Encoder i/p: string, o/p: int
def encode(s):
    _stoi = []
    for ch in s:
        _stoi.append(stoi[ch])
    return _stoi

# Decoder i/p: int, o/p: str
def decode(i:int)->str:
    _itos = []
    for _int in i:
        _itos.append(itos[_int])
    return "".join(_itos)

In [33]:
print(encode("You are being watched"))
print(decode(encode("You are being watched")))

[37, 53, 59, 1, 39, 56, 43, 1, 40, 43, 47, 52, 45, 1, 61, 39, 58, 41, 46, 43, 42]
You are being watched


In [34]:
# Encode the entire dataset
import torch 
device = "cuda" if torch.cuda.is_available() else "cpu"
data = torch.tensor(encode(text), dtype=torch.long, device=device)

In [35]:
data.shape

torch.Size([1115394])

In [36]:
data[:100]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59], device='cuda:0')

In [37]:
# Train and val set
seed = 108
split = 0.9
n = int(len(data) * split) 
# TODO: Add shuffle
# data = torch.shuffle(data, seed=seed)
train_data = data[:n]
test_data = data[n:]

In [38]:
# Max len of chunks in train data to look at
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58], device='cuda:0')

In [39]:
X = train_data[:block_size+1]
y = train_data[1:block_size+1]
print(f"X: {X}, y: {y}")
for i in range(block_size):
    context = X[:i+1]
    target = y[i]
    print(f"Input context: {context} with target:{target} ")

X: tensor([18, 47, 56, 57, 58,  1, 15, 47, 58], device='cuda:0'), y: tensor([47, 56, 57, 58,  1, 15, 47, 58], device='cuda:0')
Input context: tensor([18], device='cuda:0') with target:47 
Input context: tensor([18, 47], device='cuda:0') with target:56 
Input context: tensor([18, 47, 56], device='cuda:0') with target:57 
Input context: tensor([18, 47, 56, 57], device='cuda:0') with target:58 
Input context: tensor([18, 47, 56, 57, 58], device='cuda:0') with target:1 
Input context: tensor([18, 47, 56, 57, 58,  1], device='cuda:0') with target:15 
Input context: tensor([18, 47, 56, 57, 58,  1, 15], device='cuda:0') with target:47 
Input context: tensor([18, 47, 56, 57, 58,  1, 15, 47], device='cuda:0') with target:58 


In [40]:
train_data.shape

torch.Size([1003854])

In [41]:
# Create a dataloader
batch_size = 4
block_size = 8
# Random seed
torch.manual_seed(1337)
X = []
y = []
def get_batch(split:str):
    data = train_data if split=="train" else test_data
    # Random offsets in data
    idx = torch.randint(low=0,high=(len(data)-block_size), size=(batch_size,))
    X = torch.stack([data[i:i+block_size] for i in idx])
    y = torch.stack([data[i+1:i+1+block_size] for i in idx])
    return X, y

In [42]:
xb, yb = get_batch("train")


In [43]:
print(xb.shape)
# 4x8 -> 4 batch, 8 block size

torch.Size([4, 8])


In [44]:
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]], device='cuda:0')


In [45]:
# Bigram Model 
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size, device=device)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [46]:
#### Toy example with (B, T)
model = BigramLanguageModel()
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

#### Generate a sample
idx = torch.zeros((1, 1), dtype=torch.long, device=device) # creates a 1x1 zero tensor that starts the genration 
#### Zero is the "/n" character, it makes sense to kick off the job with it
context = model.generate(idx, max_new_tokens=100)[0] # index into 0th row to unpluck the batch dimension
print(decode(context.tolist())) # Convert to a list and decode

torch.Size([32, 65])
tensor(4.8573, device='cuda:0', grad_fn=<NllLossBackward0>)

JrTxbDkRZkNwc.wj,ZTxO-On-y$WK
baqPe?kMBFeA$G:XZSGgO-3cjMGd?gLhaGhX'YVX3tpgfNuwq&$WWv.tbaF :X3!FHaGeN


In [47]:
## Training, create a Pytorch optimizer
model = BigramLanguageModel()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [48]:
batch = 32
max_iters = 10000
eval_iters = 1000

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, y = get_batch(split)
            logits, loss = model(X, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [49]:
for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"Step: {iter}, Loss: {losses['train']:.3f}, Val Loss: {losses['val']:.3f}")
    # get a training batch
    xb, yb = get_batch("train")
    # forward pass
    logits, loss = model(xb, yb)
    # zero out the gradients
    optimizer.zero_grad()
    # backward pass
    loss.backward()
    # update parameters
    optimizer.step()
    # print out the loss
    if iter % 1000 == 0:
        print(f"Step: {iter}, Loss: {loss.item():.3f}")
    

Step: 0, Loss: 4.635, Val Loss: 4.626
Step: 0, Loss: 4.624
Step: 1000, Loss: 3.967, Val Loss: 3.976
Step: 1000, Loss: 4.096
Step: 2000, Loss: 3.511, Val Loss: 3.499
Step: 2000, Loss: 3.606
Step: 3000, Loss: 3.175, Val Loss: 3.183
Step: 3000, Loss: 3.197
Step: 4000, Loss: 2.949, Val Loss: 2.962
Step: 4000, Loss: 3.267
Step: 5000, Loss: 2.814, Val Loss: 2.821
Step: 5000, Loss: 2.871
Step: 6000, Loss: 2.704, Val Loss: 2.720
Step: 6000, Loss: 2.613
Step: 7000, Loss: 2.638, Val Loss: 2.656
Step: 7000, Loss: 2.684
Step: 8000, Loss: 2.601, Val Loss: 2.606
Step: 8000, Loss: 2.694
Step: 9000, Loss: 2.561, Val Loss: 2.573
Step: 9000, Loss: 2.768


In [50]:
# Generate a sample
idx = torch.zeros((1, 1), dtype=torch.long, device=device) # creates a 1x1 zero tensor that starts the genration 
# Zero is the "/n" character, it makes sense to kick off the job with it
context = model.generate(idx, max_new_tokens=300)[0] # index into 0th row to unpluck the batch dimension
print(decode(context.tolist())) # Convert to a list and decode


wice my.

HDERacom oroup
Yowns,MUTf isth bot mil;KI ll! ath iree s, hein lat HFtidrov ts, anend l nghir.
Trjbousel lind te l.
MIshe ce hiry:
Supr aissplle y.
H'stofu noroopetelives
MPFD:
IE:
t tthakleo W-ndo whthCeiibys the m dourivETENeshired t so mower;'se

ALk d nthrupirf s ar iris! m:

Thre inle


## Mathematical trick in self-attention
- B, T, C = 4, 8, 2 -- batch, time, channel
- X = torch.randn(B, T, C)
	- 8 tokens in a batch
	- **we want tokens to talk to each other**
	- token at t=t, should be table to talk from 0...t-1
	- from previous context to current context
		- {Weak} Average across all channels from 0...t-1 for **each batch separetely**

In [51]:
# Brute Force Approach -- Version 1
B, T, C = 4, 8, 2
x = torch.randn((B, T, C), device=device)
xbow = torch.zeros((B, T, C), device=device)
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # Dimension (t, C) where t is len(0...t-1)
        xbow[b, t] = torch.mean(xprev, dim=0) # Dimension (C,)

In [52]:
# Mathematical Trick - Refer brain/research/nlp/llm/self-atttention.md
# Vectorize the operation
torch.manual_seed(42)
a = torch.tril(torch.ones((3, 3), device=device))
a = a / torch.sum(a, dim=1, keepdim=True) # Normalizing at this stage to computer averages instead of sums
b = torch.randint(0, 10, (3, 2), device=device).float()
c=  a @ b
print("a=")
print(a)
print("b=")
print(b)
print("c=")
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]], device='cuda:0')
b=
tensor([[3., 9.],
        [8., 0.],
        [3., 7.]], device='cuda:0')
c=
tensor([[3.0000, 9.0000],
        [5.5000, 4.5000],
        [4.6667, 5.3333]], device='cuda:0')


In [53]:
# Verson 2 Context based on history, vectorized
wei = torch.tril(torch.ones((T, T), device=device))
wei = wei / torch.sum(wei, dim=1, keepdim=True) # Normalizing at this stage to computer averages instead of sums
xbow2 = wei @ x # @ is batched matrix multiply || (B, T, T) @ (B, T, C) -> (B, T, C)

In [54]:
torch.allclose(xbow, xbow2)

True

In [55]:
xbow[0], xbow2[0]

(tensor([[ 0.9980, -1.0208],
         [ 0.3580, -0.7341],
         [ 0.1487, -1.0397],
         [-0.0141, -1.1542],
         [ 0.2337, -0.8363],
         [-0.0830, -0.5064],
         [-0.1392, -0.5665],
         [-0.1603, -0.5117]], device='cuda:0'),
 tensor([[ 0.9980, -1.0208],
         [ 0.3580, -0.7341],
         [ 0.1487, -1.0397],
         [-0.0141, -1.1542],
         [ 0.2337, -0.8363],
         [-0.0830, -0.5064],
         [-0.1392, -0.5665],
         [-0.1603, -0.5117]], device='cuda:0'))

In [56]:
# Version 3 -- Weighted aggregation of past elements
tril = torch.tril(torch.ones((T, T), device=device))
wei = torch.zeros((T, T), device=device) # Affinities betwen tokens are data dependant
wei = wei.masked_fill(tril == 0, float('-inf')) # Future cannot communicate with the past
wei = F.softmax(wei, dim=1) # Normalization, get the same matrix as it exponentiates away "-inf" -> Gets normalized tril
xbow3 = wei @ x # @ is batched matrix multiply || (B, T, T) @ (B, T, C) -> (B, T, C)
torch.allclose(xbow, xbow3)

True

In [57]:
# Version 4 -- Self Attention
torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn((B, T, C), device=device)
# query vector -- what am i looking for
# key vector -- what do i contain 
# affinity -- dot(query, key) --> weights
tril = torch.tril(torch.ones((T, T), device=device))
wei = torch.zeros((T, T), device=device) # Affinities betwen tokens --> Should be data dependent --> Solved by self attention
wei = wei.masked_fill(tril == 0, float('-inf')) # Future cannot communicate with the past
wei = F.softmax(wei, dim=1) # Normalization, get the same matrix as it exponentiates away "-inf" -> Gets normalized tril
out = wei @ x # @ is batched matrix multiply || (B, T, T) @ (B, T, C) -> (B, T, C)


In [58]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]], device='cuda:0')

In [59]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]],
       device='cuda:0')

In [60]:
# Single Self attention head
torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn((B, T, C), device=device)
# query vector -- what am i looking for
# key vector -- what do i contain 
# affinity -- dot(query, key) --> weights
# Single head for self attention
head_size = 16
key = nn.Linear(C, head_size, bias=False).to(device)
query = nn.Linear(C, head_size, bias=False).to(device)
value = nn.Linear(C, head_size, bias=False).to(device) # thing that gets aggregated, x is kept private
k = key(x) # (B, T, head_size)
q = query(x) # (B, T, head_size)
# No communication hasn't happened yet ^
# Affinity
wei = q @ k.transpose(-2, -1) # (B, T, batch_size) @ (B, batch_size, T) --> (B, T, T) # Not transpose batch dimension

# wei are not zeros anymore ^ but is data dependent
# wei = torch.zeros((T, T), device=device) # Affinities betwen tokens --> Should be data dependent --> Solved by self attention

tril = torch.tril(torch.ones((T, T), device=device))
wei = wei.masked_fill(tril == 0, float('-inf')) # Future cannot communicate with the past
wei = F.softmax(wei, dim=1) # Normalization, get the same matrix as it exponentiates away "-inf" -> Gets normalized tril

v = value(x)
out = wei @ v # (B, T, T) @ (B, T, head_size) --> (B, T, head_size)
# out = wei @ x #x is private to this token


In [61]:
out.shape

torch.Size([4, 8, 16])

In [62]:
wei[0]

tensor([[1.7310e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [1.1577e-01, 1.5784e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [1.5193e-01, 6.1249e-02, 1.6173e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [3.5667e-02, 1.1354e-01, 4.5117e-02, 7.1695e-02, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [1.2516e-01, 4.6495e-01, 5.9049e-01, 3.3909e-02, 4.5423e-03, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [2.7886e-01, 1.6003e-03, 1.6429e-04, 1.8057e-01, 9.8276e-01, 3.2515e-01,
         0.0000e+00, 0.0000e+00],
        [9.6809e-02, 1.1703e-01, 4.1739e-02, 3.0337e-01, 9.0820e-03, 2.9430e-01,
         4.5407e-01, 0.0000e+00],
        [2.2710e-02, 2.2584e-01, 3.0632e-01, 4.1046e-01, 3.6135e-03, 3.8055e-01,
         5.4593e-01, 1.0000e+00]], device='cuda:0', grad_fn=<SelectBackward0>)